In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import findspark # You will need to pip install findspark
findspark.init()

In [3]:
from pyspark import SparkContext
sc = SparkContext("local", "Tutorial_02") # Spark UI at http://localhost:4040/jobs/

from pyspark.sql import SQLContext, HiveContext
sqlContext = SQLContext(sc)
#sqlContext = HiveContext(sc)

In [4]:
SOURCE = 'foosoft_sales_example.json'
df = sqlContext.read.json(SOURCE)
df.printSchema()
df.count()

root
 |-- invoice: string (nullable = true)
 |-- on_prem: double (nullable = true)
 |-- saas: double (nullable = true)
 |-- team: string (nullable = true)



40

In [5]:
df.registerTempTable('sales')

In [23]:
df_sum = sqlContext.sql('''SELECT team, 
sum(on_prem) as on_prem_sales, 
sum(saas) as saas_sales, 
(sum(on_prem + saas)) as total_sales
FROM sales 
GROUP BY team 
ORDER BY total_sales DESC''').toPandas()
df_sum

,team,on_prem_sales,saas_sales,total_sales
0,TEAM-3,614716.48,31458.27,646174.75
1,TEAM-5,390752.11,47103.48,437855.59
2,TEAM-1,403404.69,19842.66,423247.35
3,TEAM-2,328591.83,0.00,328591.83
4,TEAM-4,204476.49,0.00,204476.49


In [24]:
df_unit = sqlContext.sql('''SELECT team, 
avg(on_prem) as on_prem_avg, 
avg(saas) as saas_avg, 
(avg(on_prem + saas)) as total_avg
FROM sales 
GROUP BY team 
ORDER BY total_avg DESC''').toPandas()
df_unit

,team,on_prem_avg,saas_avg,total_avg
0,TEAM-1,57629.241429,2834.665714,60463.907143
1,TEAM-3,55883.316364,2859.842727,58743.159091
2,TEAM-2,54765.305000,0.000000,54765.305000
3,TEAM-5,43416.901111,5233.720000,48650.621111
4,TEAM-4,29210.927143,0.000000,29210.927143


In [35]:
df_sum.rank(numeric_only=True, pct=True)

,on_prem_sales,saas_sales,total_sales
0,1.0,0.8,1.0
1,0.6,1.0,0.8
2,0.8,0.6,0.6
3,0.4,0.3,0.4
4,0.2,0.3,0.2
